# MiniFlow Architecture
small tensorFlow clone

In [18]:
# imports
import numpy as np

## Generic node representation class
Each node might receive input from multiple other nodes.
each node creates a single output, which will likely be passed to other nodes.

Contains two lists:
- one to store references to the inbound ndoes.
- one to store references to the outbound nodes.

Each node will eventually calculate a value that represents its output.
- initialize the value to None to indicate that it exists but hasn't been set yet.

Each node will need to be able to pass values forward and perform backpropagation.
...

In [1]:
class Node(object):
    def __init__(self, inbound_nodes=[]):
        # Properties
        
        # Node(s) from which this Node receives values
        self.inbound_nodes = inbound_nodes
        # Node(s) to which this Node passes values
        self.outbound_nodes = []
        # A calculated value
        self.value = None
        # For each inbound Node here, add this Node as an outbound Node to _that_ Node.
        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)
        
        
        def forward(self):
            """
            Forward propagation.
            
            Compute the output value vased on `inbound_nodes` and store the result in self.value.
            """
            raise NotImplemented

## Nodes that Calculate - subclasses
While `Node` defines the base set of properties that every node holds, only specialized [subclasses](https://docs.python.org/3/tutorial/classes.html#inheritance) of `Node` will end up in the graph.

The following subclasses of `Node` will be created:
- `Input`
- `Add`

### The `Input` Subclass

In [2]:
class Input(Node):
    def __init__(self):
        # An Input node has no inbound nodes,
        # so no need to pass anything to the Node instantiator.
        Node.__init__(self)
        
    # NOTE: Input node is the only node where the value
    # may be passed as an argument to forward().
    #
    # All other node implementations should get the value
    # of the previous node from self.inbound_nodes
    #
    # Example:
    # val0 = self.inbound_nodes[0].value
    def forward(self, value=None):
        # Overwrite the value if one is passed in.
        if value is not None:
            self.value = value

Unlike the other subclasses of `Node`, the `Input` subclass does not actually calculate anything. The `Input` subclass just holds a `value`, such as a data feature or a model parameter (weight/bias).

The `value` can be either set explicitly or with the `forward()` method. This value is then fed through the rest of the neural network.

### The `Add` Subclass

`Add`, which is another subclass of `Node`, actually can perform a calculation (addition).

Use an [unpacking argument list](https://docs.python.org/3/tutorial/controlflow.html#unpacking-argument-lists) to accept any number of inputs that are summed.

In [3]:
class Add(Node):
    def __init__(self, x, y):
        # Pass the inbound_nodes list [x, y] to 
        # the Node instantiator. The Add subclass takes two
        # inbound nodes, x and y, and adds the values of those nodes.
        Node.__init__(self, [x, y])
        
    def __init__(self, *inputs):
        Node.__init__(self, inputs)
        
    def forward(self):
        """
        Set the value of this node (`self.value`) to the sum of it's inbound_nodes.
        
        """
        sum = 0
        for n in self.inbound_nodes:
            sum += n.value
        self.value = sum

### The `Mul` Subclass

`Mul`, which is another subclass of `Node`, actually can perform a calculation (multiplication).

Use an [unpacking argument list](https://docs.python.org/3/tutorial/controlflow.html#unpacking-argument-lists) to accept any number of inputs that are multiplied.

In [4]:
class Mul(Node):
    def __init__(self, x, y):
        # Pass the inbound_nodes list [x, y] to 
        # the Node instantiator. The Add subclass takes two
        # inbound nodes, x and y, and adds the values of those nodes.
        Node.__init__(self, [x, y])
        
    def __init__(self, *inputs):
        Node.__init__(self, inputs)
        
    def forward(self):
        """
        Set the value of this node (`self.value`) to multiply it's inbound_nodes.
        
        """
        product = 1
        for n in self.inbound_nodes:
            product *= n.value
        self.value = product

## Forward propagation

`MiniFlow` has two methods to help define and then run values through graphs:
- `topological_sort()`
- `forward_pass()`

The picture from Udacity shows an example of topological sorting

![An example of topological_sort](./figures/topological-sort001.jpeg)

In order to define a network, we need to define the order of operations for nodes. Given that the input to some node depends on the outputs of others, we need to flatten the graph in such a way where all the input dependencies for each node are resolved before trying to run its calculation. This is a technique called [topological sort](https://en.wikipedia.org/wiki/Topological_sorting)

The `topological_sort()` function implements topological sorting using [Kahn's Algorithm](https://en.wikipedia.org/wiki/Topological_sorting#Kahn.27s_algorithm). 

`topological_sort()` returns a sorted list of nodes in which all of the calculations can run in series.

topological_sort() takes in a feed_dict, which is how we initially set a value for an Input node. The feed_dict is represented by the Python dictionary data structure.

In [5]:
def topological_sort(feed_dict):
    """
    Sort generic nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` node and the value is the respective value feed to that node.

    Returns a list of sorted nodes.
    """

    input_nodes = [n for n in feed_dict.keys()]
    #import pdb
    #pdb.set_trace()

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L

`forward_pass()` actually runs the network and outputs a value.

In [6]:
def forward_pass(output_node, sorted_nodes):
    """
    Performs a forward pass through a list of sorted nodes.

    Arguments:

        `output_node`: The output node of the graph (no outgoing edges).
        `sorted_nodes`: a topologically sorted list of nodes.

    Returns the output node's value
    """

    for n in sorted_nodes:
        n.forward()

    return output_node.value

In [7]:
"""
This script builds and runs a graph with miniflow.

"""

x, y, z = Input(), Input(), Input()

fAdd = Add(x, y, z)
fMul = Mul(x, y, z)
f = Add(fAdd, fMul)

feed_dict = {x: 4, y: 5, z: 10}


sorted_nodes = topological_sort(feed_dict)
output = forward_pass(f, sorted_nodes)

# NOTE: because topological_sort set the values for the `Input` nodes we could also access
# the value for x with x.value (same goes for y).
print("{} + {} + {} = {} (according to miniflow)".format(feed_dict[x], feed_dict[y], feed_dict[z], output))

4 + 5 + 10 = 219 (according to miniflow)


# Learning and Loss

## The linear function

The picture from Udacity shows a neuron

![Linear function neuron](./figures/neuron.png)

The linear function `Node` aka Neuron depends on three components:
- inputs $x_i$,
- weights $\omega_i$
- bias $b$

The output $y$, is just the weighted sum of the inputs plus the bias.

$$
y = \sum_i x_i \omega_i + b
$$

by varying the weights, you can vary the amount of influence any given input has on the output. The learning aspect of neural networks takes place during a process known as backpropagation. In backpropogation, the network modifies the weights to improve the network's output accuracy.


Create the linear function `Node` by setting `self.value` to the bias and then loop through the inputs and weights, adding each weighted input to `self.value`. Notice calling `.value` on `self.inbound_nodes[0]` or `self.inbound_nodes[1]` returns a list.

The [zip method](https://docs.python.org/3/library/functions.html#zip) was used to get iterators from the iterables inputs and weights. 

    zip(iter1 [,iter2 [...]]) --> zip object
    
Return a zip object whose `.__next__()` method returns a tuple where
the i-th element comes from the i-th iterable argument.  The `.__next__()`
method continues until the shortest iterable in the argument sequence
is exhausted and then it raises StopIteration.

We use `numpy.array` ([documentation](https://docs.scipy.org/doc/numpy/reference/generated/numpy.array.html)) to handle matrices and vectors as input to the `Linear` node. For this `np.dot` will be used to multiply 2D arrays ([documentation](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html)). `numpy` also overloads the `__add__` operator so two arrays can be added directly (eg. `np.array() + np.array()`)

In [24]:
class Linear(Node):
    def __init__(self, X, W, b):
        Node.__init__(self, [X, W, b])

        # NOTE: The weights and bias properties here are not
        # numbers, but rather references to other nodes.
        # The weight and bias values are stored within the
        # respective nodes.

    def forward(self):
        """
        Set self.value to the value of the linear function output.
        
        """
        # for implementation details see below (section Transform)
        X = self.inbound_nodes[0].value
        W = self.inbound_nodes[1].value
        b = self.inbound_nodes[2].value
        self.value = np.dot(X, W) + b
        
        #import pdb 
        #pdb.set_trace()
        #sum = 0
        #for i, x in enumerate(self.inbound_nodes[0].value):
        #    weight = self.inbound_nodes[1].value
        #    sum += x*weight[i]
        #b = self.inbound_nodes[2].value
        #self.value = sum + b
        
        #inputs = self.inbound_nodes[0].value
        #weights = self.inbound_nodes[1].value
        #bias = self.inbound_nodes[2]
        #self.value = bias.value
        #for x, w in zip(inputs, weights):
        #    self.value += x * w

In [25]:
"""
NOTE: Here an Input node is used for more than a scalar.
In the case of weights and inputs the value of the Input node is a python list!

In general, there's no restriction on the values that can be passed to an Input node.
"""

inputs, weights, bias = Input(), Input(), Input()

f = Linear(inputs, weights, bias)

feed_dict = {
    inputs: [6, 14, 3],
    weights: [0.5, 0.25, 1.4],
    bias: 2
}

graph = topological_sort(feed_dict)
output = forward_pass(f, graph)

print(output) # should be 12.7 with this example

12.7


## Transform

![Linear function neuron](./figures/transform.png)

Linear algebra nicely reflects the idea of transforming values between layers in a graph. In fact, the concept of a transform does exactly what a layer should do - it converts inputs to outputs in many dimensions.

The output equation for $y$, was the weighted sum of the inputs plus the bias.

$$
y = \sum_i x_i \omega_i + b
$$

Now the input and the weights are going to be a matrices X and W respectively. The bias b is now a vector instead of a scalar.

Some examples on the structure of $X$ and $W$ using the `Linear` node:

(In this context an input/output is synonymous with a feature.

- If we have a node with one input and k outpus (mapping one input to k outputs) then:

    $X$ is a $1 \times 1$ matrix

    $$
    X = [X_{11}]
    $$
    $$
    1 \times 1 \matrix{ matrix, one element.}
    $$
    
    $W$ becomes a $1 \times k$ matrix (a row vector)
    
    $$
    W = \begin{bmatrix} W_{11} & W_{12} & W_{13} & \ldots & W_{1k} \end{bmatrix}
    $$
    $$
    1 \times k \text{ weights matrix, row matrix/vector.}
    $$
    
    The result of the matrix multiplication of $X$ and $W$ is a $1 \times k$ matrix. Since $b$ is also a $1 \times k$ row matrix (1 bias per output), $b$ is added to the output of the $X$ and $W$ matrix multiplication.
    
    
- If we map n inputs to k outpus (mapping multiple inputs to k outputs) then:

    $X$ is now a $1 \times n$ matrix and $W$ is a $n \times k$ matrix. The result of the matrix multiplication is still a $1 \times k$ matrix so the use of the biases remain the same.
    
    $$
    X = \begin{bmatrix} X_{11} & X_{12} & X_{13} & \ldots & X_{1n} \end{bmatrix}
    $$
    $$
    1 \times n \text{ input matrix, n inputs/features.}
    $$
    
    $$
    W = \begin{bmatrix} W_{11} & W_{12} & W_{13} & \ldots & W_{1k} \\
                        W_{21} & W_{22} & W_{23} & \ldots & W_{2k} \\
                        W_{31} & W_{32} & W_{33} & \ldots & W_{3k} \\
                        \ldots &        &        & \ddots &        \\
                        W_{n1} & W_{n2} & W_{n3} & \ldots & W_{nk} \\
    \end{bmatrix}
    $$
    $$
    1 \times k \text{ weights matrix, row matrix/vector.}
    $$
    
    $$
    b = \begin{pmatrix} b_{1} & b_{2} & X_{3} & \ldots & b_{k} \end{pmatrix}
    $$
    $$
    1 \times k \text{row vector of biases, one for each output.}
    $$

An example of n inputs would be a 28px by 28px greyscale image, as is in the case of images in the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, a set of handwritten digits. We can reshape the image such that it's a 1 by $784$ matrix, $n = 784$. Each pixel is an input/feature.

In practice, it's common to feed in multiple data examples in each forward pass rather than just one. The reasoning for this is the examples can be processed in parallel, resulting in big performance gains. The number of examples is called the batch size. Common numbers for the batch size are 32, 64, 128, 256, 512. Generally, it's the most we can comfortably fit in memory.

This means that $X$ becomes a $m \times n$ matrix and $W$ and $b$ remain the same. The result of the matrix multiplication is now $m \times k$, so the addition of $b$ is [broadcast](https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html) over each row.

$$
X = \begin{bmatrix} X_{11} & X_{12} & X_{13} & \ldots & X_{1n} \\
                    X_{21} & X_{22} & X_{23} & \ldots & X_{2n} \\
                    X_{31} & X_{32} & X_{33} & \ldots & X_{3n} \\
                    \ldots &        &        & \ddots &        \\
                    X_{m1} & X_{m2} & X_{m3} & \ldots & X_{mn} \\
\end{bmatrix}
$$
$$
X\text{ is now an } m \times n \text{ matrix. Each row has n inputs/features.}
$$

In the context of MNIST each row of $X$ is an image reshaped from $28$ by $28$ to $1 \times 784$.


$$
Z = XW + b
$$

This equation can also be views as $Z = XW + B$ where $B$ is the biases vector, $b$, stacked $m$ times as a row. Due to broadcasting it's abbreviated to $Z = XW + b$.

In [26]:
"""
Code test section
"""

X, W, b = Input(), Input(), Input()

f = Linear(X, W, b)

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2., -3], [2., -3]])
b_ = np.array([-3., -5])

feed_dict = {X: X_, W: W_, b: b_}

graph = topological_sort(feed_dict)
output = forward_pass(f, graph)

"""
Output should be:
[[-9., 4.],
[-9., 4.]]
"""
print(output)

[[-9.  4.]
 [-9.  4.]]


# Sigmoid Function

Neural networks take advantage of alternating transforms and activation functions to better categorize outputs. The sigmoid function is among the most common activation functions.

$$
sigmoid(x) = \sigma(x) = \frac{1}{1+e^{-x}}
$$

![logistic curve](./figures/logistic-curve.svg)
Graph of the sigmoid funciton. Notice the "S" shape.

Linear transforms are great for simply shifting values, but neural networks often require a more nuanced transform. For instance, one of the original designs for an artificial neuron, the [perceptron](https://en.wikipedia.org/wiki/Perceptron), exhibit binary output behavior. Perceptrons compare a weighted input to a threshold. When the weighted input exceeds the threshold, the perceptron is **activated** and outputs `1`, otherwise it outputs `0`.

The behavior of a perceptron can be modeled as a step function

![step function](./figures/step.png)
Example of a step function (The jump between $y = 0$ and $y = 1$ should be instantaneous).

Activation, the idea of binary output behavior, generally makes sense for classification problems. For example, if you ask the network to hypothesize if a handwritten image is a '9', you're effectively asking for a binary output - yes, this is a '9', or no, this is not a '9'. A step function is the starkest form of a binary output, which is great, but step functions are not continuous and not differentiable, which is very bad. Differentiation is what makes gradient descent possible.

The sigmoid function $\sigma(x)$, replaces thresholding with a beautiful S-shaped curve (shown above) that mimics the activation behavior of a perceptron while maintaining continuity, and thus differentiability. As a bonus, the sigmoid function has a very simple derivative that looks remarkably similar to the sigmoid itself.

$$
\sigma'(x) = \sigma(x) \cdot (1 - \sigma(x))
$$

Notice that the sigmoid function only has one parameter. Remember that sigmoid is an activation function (non-linearity), meaning it takes a single input and performs a mathematical operation on it.

Conceptually, the sigmoid function makes decisions. When given weighted features from some data, it indicates whether or not the features contribute to a classification. In that way, a sigmoid activation works well following a linear transformation. As it stands right now with random weights and bias, the sigmoid node's output is also random. The process of learning through backpropagation and gradient descent, modifies the weights and bias such that activation of the sigmoid node begins to match expected outputs.
To add the equation of the sigmoid function to the `MiniFlow` library we use `np.exp` ([documentation](https://docs.scipy.org/doc/numpy/reference/generated/numpy.exp.html)).

We use `Sigmoid` in conjunction with `Linear`

In [40]:
class Sigmoid(Node):
    """
    You need to fix the `_sigmoid` and `forward` methods.
    """
    def __init__(self, node):
        Node.__init__(self, [node])

    def _sigmoid(self, x):
        """
        This method is separate from `forward` because it
        will be used later with `backward` as well.

        `x`: A numpy array-like object.

        Return the result of the sigmoid function.

        """
        return 1. / (1. + np.exp(-x))
        

    def forward(self):
        """
        Set the value of this node to the result of the
        sigmoid function, `_sigmoid`.

        """
        x = self.inbound_nodes[0].value
        self.value = self._sigmoid(x)

In [41]:
"""
This network feeds the output of a linear transform
to the sigmoid function.

Finish implementing the Sigmoid class in miniflow.py!

"""

X, W, b = Input(), Input(), Input()

f = Linear(X, W, b)
g = Sigmoid(f)

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2., -3], [2., -3]])
b_ = np.array([-3., -5])

feed_dict = {X: X_, W: W_, b: b_}

graph = topological_sort(feed_dict)
output = forward_pass(g, graph)

"""
Output should be:
[[  1.23394576e-04   9.82013790e-01]
 [  1.23394576e-04   9.82013790e-01]]
"""
print(output)


[[  1.23394576e-04   9.82013790e-01]
 [  1.23394576e-04   9.82013790e-01]]


# Cost

Because the derivative of the sigmoid function, the sigmoid function is actually a part of its own derivative. Keeping _sigmoid separate means we won't have to implement it twice for forward and backward propagations.

We have used weights and biases to compute outputs. And we used an activation function to categorize the output. Neural networks improve the accuracy of their outputs by modifying weights and biases in response to training against labeled datasets.
There are many techniques for defining the accuracy of a neural network, all of which center on the network's ability to produce values that come as close as possible to known correct values. People use different names for this accuracy measurement, often terming it loss or cost.

We will calculate the cost using the mean squared error (MSE):

$$
C(w, b) = \frac{1}{m} \sum_x (y(x) - a)^2
$$

Here $w$ denotes the collection of all weights in the network, $b$ all the biases, m is the total number of training examples, $a$ is the approximation of $y(x)$ by the network, both a and $y(x)$ are vectors of the same length.

The collection of weights is all the weight matrices flattened into vectors and concatenated to one big vector. The same goes for the collection of biases except they're already vectors so there's no need to flatten them prior to the concatenation.

Here's an example of creating w in code:

In [43]:
# 2 by 2 matrices
w1  = np.array([[1, 2], [3, 4]])
w2  = np.array([[5, 6], [7, 8]])

# flatten
w1_flat = np.reshape(w1, -1)
w2_flat = np.reshape(w2, -1)

w = np.concatenate((w1_flat, w2_flat))
# array([1, 2, 3, 4, 5, 6, 7, 8])
w

array([1, 2, 3, 4, 5, 6, 7, 8])

It's a nice way to abstract all the weights and biases used in the neural network and makes some things easier to write as we'll see soon in the upcoming gradient descent sections.

The cost, $C$, depends on the difference between the correct output, $y(x)$, and the network's output, $a$. It's easy to see that no difference between $y(x)$ and $a$ (for all values of $x$) leads to a cost of 0.

This is the ideal situation, and in fact the learning process revolves around minimizing the cost as much as possible.

As it stands right now, it outputs gibberish. The activation of the sigmoid node means nothing because the network has no labeled output against which to compare. Furthermore, the weights and bias cannot change and learning cannot happen without a cost.

We implement the cost function using the `MSE` method ([link](https://en.wikipedia.org/wiki/Mean_squared_error)) and `np.square` ([documentation](https://docs.scipy.org/doc/numpy/reference/generated/numpy.square.html)). To reshape the weights we use reshape ([documentation](https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html))

In [44]:
class MSE(Node):
    def __init__(self, y, a):
        """
        The mean squared error cost function.
        Should be used as the last node for a network.
        """
        # Call the base class' constructor.
        Node.__init__(self, [y, a])

    def forward(self):
        """
        Calculates the mean squared error.
        """
        # NOTE: We reshape these to avoid possible matrix/vector broadcast
        # errors.
        #
        # For example, if we subtract an array of shape (3,) from an array of shape
        # (3,1) we get an array of shape(3,3) as the result when we want
        # an array of shape (3,1) instead.
        #
        # Making both arrays (3,1) insures the result is (3,1) and does
        # an elementwise subtraction as expected.
        y = self.inbound_nodes[0].value.reshape(-1, 1)
        a = self.inbound_nodes[1].value.reshape(-1, 1)
        
        diff = y - a
        self.value = np.mean(diff**2) 

In [47]:
"""
Test MSE method with this script!

"""

y, a = Input(), Input()
cost = MSE(y, a)

y_ = np.array([1, 2, 3])
a_ = np.array([4.5, 5, 10])

feed_dict = {y: y_, a: a_}
graph = topological_sort(feed_dict)
# forward pass
forward_pass(cost, graph)

"""
Expected output

23.4166666667
"""
print(cost.value)

23.4166666667


The math behind MSE reflects the equation of the cost function above, where $y$ is target output and $a$ is output computed by the neural network. We then square the difference `diff**2`, alternatively, this could be `np.square(diff)`. Lastly we need to sum the squared differences and divide by the total number of examples m. This can be achieved in with `np.mean` or `(1 /m) * np.sum(diff**2)`.

Note the order of $y$ and $a$ doesn't actually matter, we could switch them around $(a - y)$ and get the same value.

# Gradient Descent